# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # Just go through the text and transform it.
    source_id_text = []
    source_lines = source_text.split('\n')
    for idx, line in enumerate(source_lines):
        source_id_text.append([])
        for word in line.split():
            source_id_text[idx].append(source_vocab_to_int[word])
        
    target_id_text = []
    target_lines = target_text.split('\n')
    for idx, line in enumerate(target_lines):
        target_id_text.append([])
        for word in line.split():
            target_id_text[idx].append(target_vocab_to_int[word])
        target_id_text[idx].append(target_vocab_to_int['<EOS>'])
    #print(source_id_text)
    #print(target_id_text)
    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)


Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.1
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # tf.float32
    input_var = tf.placeholder(tf.int32,[None,None],"input")
    targets = tf.placeholder(tf.int32,[None,None])
    learning_rate = tf.placeholder(tf.float32,name = "learning_rate")
    keep_probability = tf.placeholder(tf.float32,name ="keep_prob",)
    target_sequence_length = tf.placeholder(tf.int32,[None],"target_sequence_length")
    max_target_sequence_length = tf.reduce_max(target_sequence_length,name="max_target_len")
    source_sequence_length = tf.placeholder(tf.int32,[None],"source_sequence_length")
    return input_var, targets, learning_rate, keep_probability, target_sequence_length, max_target_sequence_length, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start\n    ioloop.IOLoop.instance().start()', 'File "/usr/local/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start\n    super(ZMQIOLoop, self).start()', 'F

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:

def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    #print(target_data)
    # TODO: Implement Function
    target_int = target_vocab_to_int['<GO>']
    go_id = tf.constant(target_int, shape=(batch_size,1), dtype=tf.int32)
    # Concatenate the vector without the last word id with the go ids vector
    processed_input = tf.concat([go_id,target_data[:,:-1]],1)
    return processed_input
    
   

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    cells = []
    for x in range(num_layers):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cells.append(lstm_drop)
    lstms = tf.contrib.rnn.MultiRNNCell(cells)
    embed_enc = tf.contrib.layers.embed_sequence(rnn_inputs, vocab_size = source_vocab_size, embed_dim = encoding_embedding_size)
    output, state = tf.nn.dynamic_rnn(lstms, embed_enc, source_sequence_length, dtype=tf.float32)
    return (output, state)
    
   

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    th = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    dec = tf.contrib.seq2seq.BasicDecoder(dec_cell, th, encoder_state, output_layer)
    final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(dec,maximum_iterations=max_summary_length)
    return final_outputs
   


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    s_ids = tf.tile([start_of_sequence_id], [batch_size]) # ??
    # Create the embedding helper.
    g_emb_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, s_ids, end_of_sequence_id)
    dec = tf.contrib.seq2seq.BasicDecoder(dec_cell, g_emb_helper, encoder_state, output_layer)
    final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(dec,maximum_iterations=max_target_sequence_length)
    return final_outputs
    #return None



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    lstms = []
    for x in range(num_layers):
        lstm = tf.contrib.rnn.LSTMCell(rnn_size)
        lstm_drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        lstms.append(lstm_drop)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell(lstms)
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    k_init = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
    dense_layer = Dense(target_vocab_size,kernel_initializer = k_init)
    
    
    
    with tf.variable_scope("dec") as scope:
        tr_decoder_output = decoding_layer_train(
            encoder_state, stacked_lstm, dec_embed_input, 
            target_sequence_length, max_target_sequence_length, 
            dense_layer, keep_prob)
        scope.reuse_variables()
        
        target_int_go = target_vocab_to_int['<GO>']
        target_int_eos = target_vocab_to_int['<EOS>']
        
        inf_decoder_output = decoding_layer_infer(
            encoder_state, stacked_lstm, dec_embeddings, 
            target_int_go, target_int_eos, 
            max_target_sequence_length, target_vocab_size, 
            dense_layer, batch_size, keep_prob)
    
    return tr_decoder_output, inf_decoder_output
    # TODO: Implement Function
    
    
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    output, state = encoding_layer(input_data, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    decoder_output_tr, decoder_output_inf = decoding_layer(processed_input, state,
                   target_sequence_length, max_target_sentence_length,
                   rnn_size, num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, dec_embedding_size)
    
    return decoder_output_inf, decoder_output_inf



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 25 # 20
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 128
decoding_embedding_size = 128
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = True


### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/269 - Train Accuracy: 0.2354, Validation Accuracy: 0.3117, Loss: 5.7253
Epoch   0 Batch    2/269 - Train Accuracy: 0.2676, Validation Accuracy: 0.3117, Loss: 5.5313
Epoch   0 Batch    3/269 - Train Accuracy: 0.2460, Validation Accuracy: 0.3105, Loss: 5.3848
Epoch   0 Batch    4/269 - Train Accuracy: 0.2320, Validation Accuracy: 0.3101, Loss: 5.2189
Epoch   0 Batch    5/269 - Train Accuracy: 0.2325, Validation Accuracy: 0.3096, Loss: 5.0372
Epoch   0 Batch    6/269 - Train Accuracy: 0.2785, Validation Accuracy: 0.3096, Loss: 4.7093
Epoch   0 Batch    7/269 - Train Accuracy: 0.2765, Validation Accuracy: 0.3096, Loss: 4.5774
Epoch   0 Batch    8/269 - Train Accuracy: 0.2533, Validation Accuracy: 0.3215, Loss: 4.6306
Epoch   0 Batch    9/269 - Train Accuracy: 0.2797, Validation Accuracy: 0.3224, Loss: 4.4398
Epoch   0 Batch   10/269 - Train Accuracy: 0.2639, Validation Accuracy: 0.3383, Loss: 4.5258
Epoch   0 Batch   11/269 - Train Accuracy: 0.3054, Validation Accuracy

Epoch   0 Batch   90/269 - Train Accuracy: 0.4229, Validation Accuracy: 0.4795, Loss: 2.6816
Epoch   0 Batch   91/269 - Train Accuracy: 0.4656, Validation Accuracy: 0.4917, Loss: 2.5255
Epoch   0 Batch   92/269 - Train Accuracy: 0.4689, Validation Accuracy: 0.4902, Loss: 2.5291
Epoch   0 Batch   93/269 - Train Accuracy: 0.4688, Validation Accuracy: 0.4708, Loss: 2.4370
Epoch   0 Batch   94/269 - Train Accuracy: 0.4706, Validation Accuracy: 0.4926, Loss: 2.5231
Epoch   0 Batch   95/269 - Train Accuracy: 0.4707, Validation Accuracy: 0.4927, Loss: 2.4927
Epoch   0 Batch   96/269 - Train Accuracy: 0.4547, Validation Accuracy: 0.4836, Loss: 2.5008
Epoch   0 Batch   97/269 - Train Accuracy: 0.4682, Validation Accuracy: 0.4886, Loss: 2.4794
Epoch   0 Batch   98/269 - Train Accuracy: 0.4841, Validation Accuracy: 0.4965, Loss: 2.4498
Epoch   0 Batch   99/269 - Train Accuracy: 0.4434, Validation Accuracy: 0.4923, Loss: 2.6273
Epoch   0 Batch  100/269 - Train Accuracy: 0.4918, Validation Accuracy

Epoch   0 Batch  179/269 - Train Accuracy: 0.5293, Validation Accuracy: 0.5491, Loss: 1.9918
Epoch   0 Batch  180/269 - Train Accuracy: 0.5289, Validation Accuracy: 0.5433, Loss: 1.9869
Epoch   0 Batch  181/269 - Train Accuracy: 0.5300, Validation Accuracy: 0.5445, Loss: 1.9980
Epoch   0 Batch  182/269 - Train Accuracy: 0.5362, Validation Accuracy: 0.5537, Loss: 2.0024
Epoch   0 Batch  183/269 - Train Accuracy: 0.5916, Validation Accuracy: 0.5467, Loss: 1.7432
Epoch   0 Batch  184/269 - Train Accuracy: 0.5105, Validation Accuracy: 0.5502, Loss: 2.0639
Epoch   0 Batch  185/269 - Train Accuracy: 0.5483, Validation Accuracy: 0.5528, Loss: 1.9268
Epoch   0 Batch  186/269 - Train Accuracy: 0.5126, Validation Accuracy: 0.5502, Loss: 2.0548
Epoch   0 Batch  187/269 - Train Accuracy: 0.5372, Validation Accuracy: 0.5462, Loss: 1.9165
Epoch   0 Batch  188/269 - Train Accuracy: 0.5421, Validation Accuracy: 0.5533, Loss: 1.9113
Epoch   0 Batch  189/269 - Train Accuracy: 0.5421, Validation Accuracy

Epoch   1 Batch    1/269 - Train Accuracy: 0.5495, Validation Accuracy: 0.5787, Loss: 1.8361
Epoch   1 Batch    2/269 - Train Accuracy: 0.5601, Validation Accuracy: 0.5844, Loss: 1.8074
Epoch   1 Batch    3/269 - Train Accuracy: 0.5574, Validation Accuracy: 0.5806, Loss: 1.8308
Epoch   1 Batch    4/269 - Train Accuracy: 0.5138, Validation Accuracy: 0.5469, Loss: 1.8710
Epoch   1 Batch    5/269 - Train Accuracy: 0.5291, Validation Accuracy: 0.5692, Loss: 1.9073
Epoch   1 Batch    6/269 - Train Accuracy: 0.5847, Validation Accuracy: 0.5858, Loss: 1.6947
Epoch   1 Batch    7/269 - Train Accuracy: 0.5743, Validation Accuracy: 0.5905, Loss: 1.7826
Epoch   1 Batch    8/269 - Train Accuracy: 0.5378, Validation Accuracy: 0.5683, Loss: 1.8278
Epoch   1 Batch    9/269 - Train Accuracy: 0.5533, Validation Accuracy: 0.5660, Loss: 1.7903
Epoch   1 Batch   10/269 - Train Accuracy: 0.5541, Validation Accuracy: 0.5848, Loss: 1.8168
Epoch   1 Batch   11/269 - Train Accuracy: 0.5654, Validation Accuracy

Epoch   1 Batch   90/269 - Train Accuracy: 0.5461, Validation Accuracy: 0.5821, Loss: 1.7633
Epoch   1 Batch   91/269 - Train Accuracy: 0.5832, Validation Accuracy: 0.5884, Loss: 1.6323
Epoch   1 Batch   92/269 - Train Accuracy: 0.5951, Validation Accuracy: 0.6028, Loss: 1.6208
Epoch   1 Batch   93/269 - Train Accuracy: 0.6088, Validation Accuracy: 0.6041, Loss: 1.5825
Epoch   1 Batch   94/269 - Train Accuracy: 0.5790, Validation Accuracy: 0.5855, Loss: 1.6499
Epoch   1 Batch   95/269 - Train Accuracy: 0.5778, Validation Accuracy: 0.5820, Loss: 1.6339
Epoch   1 Batch   96/269 - Train Accuracy: 0.5861, Validation Accuracy: 0.6012, Loss: 1.6545
Epoch   1 Batch   97/269 - Train Accuracy: 0.5901, Validation Accuracy: 0.6048, Loss: 1.6389
Epoch   1 Batch   98/269 - Train Accuracy: 0.5997, Validation Accuracy: 0.6002, Loss: 1.6072
Epoch   1 Batch   99/269 - Train Accuracy: 0.5604, Validation Accuracy: 0.5946, Loss: 1.7298
Epoch   1 Batch  100/269 - Train Accuracy: 0.6031, Validation Accuracy

...

Epoch  23 Batch  179/269 - Train Accuracy: 0.9688, Validation Accuracy: 0.9711, Loss: 0.2272
Epoch  23 Batch  180/269 - Train Accuracy: 0.9749, Validation Accuracy: 0.9711, Loss: 0.2027
Epoch  23 Batch  181/269 - Train Accuracy: 0.9668, Validation Accuracy: 0.9716, Loss: 0.2247
Epoch  23 Batch  182/269 - Train Accuracy: 0.9658, Validation Accuracy: 0.9709, Loss: 0.2027
Epoch  23 Batch  183/269 - Train Accuracy: 0.9798, Validation Accuracy: 0.9696, Loss: 0.1479
Epoch  23 Batch  184/269 - Train Accuracy: 0.9652, Validation Accuracy: 0.9688, Loss: 0.2244
Epoch  23 Batch  185/269 - Train Accuracy: 0.9736, Validation Accuracy: 0.9692, Loss: 0.1906
Epoch  23 Batch  186/269 - Train Accuracy: 0.9792, Validation Accuracy: 0.9703, Loss: 0.1844
Epoch  23 Batch  187/269 - Train Accuracy: 0.9697, Validation Accuracy: 0.9708, Loss: 0.1994
Epoch  23 Batch  188/269 - Train Accuracy: 0.9720, Validation Accuracy: 0.9699, Loss: 0.1848
Epoch  23 Batch  189/269 - Train Accuracy: 0.9744, Validation Accuracy

Epoch  24 Batch    1/269 - Train Accuracy: 0.9740, Validation Accuracy: 0.9706, Loss: 0.1792
Epoch  24 Batch    2/269 - Train Accuracy: 0.9716, Validation Accuracy: 0.9698, Loss: 0.2143
Epoch  24 Batch    3/269 - Train Accuracy: 0.9729, Validation Accuracy: 0.9713, Loss: 0.1821
Epoch  24 Batch    4/269 - Train Accuracy: 0.9652, Validation Accuracy: 0.9701, Loss: 0.2265
Epoch  24 Batch    5/269 - Train Accuracy: 0.9747, Validation Accuracy: 0.9693, Loss: 0.1996
Epoch  24 Batch    6/269 - Train Accuracy: 0.9734, Validation Accuracy: 0.9683, Loss: 0.1690
Epoch  24 Batch    7/269 - Train Accuracy: 0.9697, Validation Accuracy: 0.9669, Loss: 0.2070
Epoch  24 Batch    8/269 - Train Accuracy: 0.9709, Validation Accuracy: 0.9666, Loss: 0.2080
Epoch  24 Batch    9/269 - Train Accuracy: 0.9735, Validation Accuracy: 0.9650, Loss: 0.1857
Epoch  24 Batch   10/269 - Train Accuracy: 0.9715, Validation Accuracy: 0.9645, Loss: 0.1886
Epoch  24 Batch   11/269 - Train Accuracy: 0.9686, Validation Accuracy

Epoch  24 Batch   90/269 - Train Accuracy: 0.9661, Validation Accuracy: 0.9669, Loss: 0.2115
Epoch  24 Batch   91/269 - Train Accuracy: 0.9770, Validation Accuracy: 0.9670, Loss: 0.1859
Epoch  24 Batch   92/269 - Train Accuracy: 0.9694, Validation Accuracy: 0.9690, Loss: 0.2037
Epoch  24 Batch   93/269 - Train Accuracy: 0.9741, Validation Accuracy: 0.9707, Loss: 0.2121
Epoch  24 Batch   94/269 - Train Accuracy: 0.9688, Validation Accuracy: 0.9698, Loss: 0.2057
Epoch  24 Batch   95/269 - Train Accuracy: 0.9702, Validation Accuracy: 0.9635, Loss: 0.1884
Epoch  24 Batch   96/269 - Train Accuracy: 0.9616, Validation Accuracy: 0.9627, Loss: 0.2247
Epoch  24 Batch   97/269 - Train Accuracy: 0.9681, Validation Accuracy: 0.9641, Loss: 0.2150
Epoch  24 Batch   98/269 - Train Accuracy: 0.9701, Validation Accuracy: 0.9640, Loss: 0.2000
Epoch  24 Batch   99/269 - Train Accuracy: 0.9725, Validation Accuracy: 0.9671, Loss: 0.2043
Epoch  24 Batch  100/269 - Train Accuracy: 0.9706, Validation Accuracy

Epoch  24 Batch  179/269 - Train Accuracy: 0.9754, Validation Accuracy: 0.9716, Loss: 0.1837
Epoch  24 Batch  180/269 - Train Accuracy: 0.9737, Validation Accuracy: 0.9710, Loss: 0.1866
Epoch  24 Batch  181/269 - Train Accuracy: 0.9705, Validation Accuracy: 0.9718, Loss: 0.2157
Epoch  24 Batch  182/269 - Train Accuracy: 0.9668, Validation Accuracy: 0.9701, Loss: 0.1914
Epoch  24 Batch  183/269 - Train Accuracy: 0.9802, Validation Accuracy: 0.9676, Loss: 0.1397
Epoch  24 Batch  184/269 - Train Accuracy: 0.9632, Validation Accuracy: 0.9672, Loss: 0.2281
Epoch  24 Batch  185/269 - Train Accuracy: 0.9730, Validation Accuracy: 0.9686, Loss: 0.1956
Epoch  24 Batch  186/269 - Train Accuracy: 0.9738, Validation Accuracy: 0.9698, Loss: 0.1886
Epoch  24 Batch  187/269 - Train Accuracy: 0.9719, Validation Accuracy: 0.9714, Loss: 0.1947
Epoch  24 Batch  188/269 - Train Accuracy: 0.9754, Validation Accuracy: 0.9708, Loss: 0.1675
Epoch  24 Batch  189/269 - Train Accuracy: 0.9789, Validation Accuracy

Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    word_ids = []
    words = sentence.lower().split()
    for word in words:
        if word in vocab_to_int:
            word_ids.append(vocab_to_int[word])
        else:
            word_ids.append(vocab_to_int['<UNK>'])
    return word_ids
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [31]:
translate_sentence = 'he saw a old yellow car .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [195, 225, 140, 86, 133, 110, 73]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'car', '.']

Prediction
  Word Ids:      [5, 11, 144, 289, 219, 28, 1]
  French Words: il vu une vieille voiture . <EOS>


In [ ]:
#   il vu une vieille voiture  => he saw an old car (google translate)

## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.